# DIT Natural Language Processing lesson 2025

## Using character-level representations for generation




The model was trained in lesson 19. Now we simply load it and model and run it

In [ ]:
# Importing the dependencies

import nltk
import numpy as np
import random
import sys

from keras.models import model_from_json
from nltk.corpus import gutenberg

In [ ]:
# This max len can be modified, because this is an LSTM
MAXLEN = 40

# Files with network architecture and weights
MODEL_JSON = "shakes_lstm_model.json"
MODEL_WEIGHTS = "shakes_lstm_5.weights.h5"

Loading the _text_ to serve as seed (prompt), as well as the dictionaries to codify the input and decodify the output

In [ ]:
nltk.download('gutenberg')

In [ ]:
text = ''
for txt in gutenberg.fileids():
    if 'shakespeare' in txt:
        text += gutenberg.raw(txt).lower()
chars = sorted(list(set(text)))

# dictionary from character to index
char_indices = dict((c, i) for i, c in enumerate(chars))

# distionary from index to character
indices_char = dict((i, c) for i, c in enumerate(chars))

print('corpus length: {} total chars: {}'.format(len(text), len(chars)))

print(text[:500])
print(char_indices)
print(indices_char)

**Loading the pre-trained network**

The structure and weight files have to be loaded beforehand.


In [ ]:
with open(MODEL_JSON, "r") as json_file:
    json_string = json_file.read()
model = model_from_json(json_string)
model.load_weights(MODEL_WEIGHTS)

model.summary()

**Temperature**

temperature > 1 : more diverse outcome

temperature < 1 : more strict (try to "copy")

In [ ]:
def sample(preds, temperature=1.0):
    """Sampler to generate character sequences"""
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    # produces a number of random outcomes,
    # given a probability distribution
    # n=1    number of experiments
    # preds  sequence of probabilities
    # size=1
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
start_index = random.randint(0, len(text) - MAXLEN - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)
    # Getting a random starting text
    sentence = text[start_index: start_index + MAXLEN]
    generated = ''
    generated += sentence

    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    for i in range(400):
        # one-hot representation
        x = np.zeros((1, MAXLEN, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        # Producing the prediction
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)

        # looking up the next character and adding it
        next_char = indices_char[next_index]
        generated += next_char

        #updating the seed
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()  # to display it right away
    print()

# lower values should look "more Shakesperean"

## Homework

1. Try with shorter/longer contextual sequences
2. Build a model that tries to mimic Dante

**End of the notebook**